In [1]:
# Libraries
import datetime
import os
import sqlite3
import pandas as pd
import os
from platform import python_version
from datetime import datetime

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Show current working directory
print(os.getcwd())

c:\ZHAW\Sc_Programming\Project\Github\sp_project_janosh_marius\backend_sp_db


In [3]:
exportdir = ['Imports', 'Exports', 'Database']


for exportdirs in exportdir:
    output_dir = os.path.join(r'C:\ZHAW\Sc_Programming\Project\Github\sp_project_janosh_marius\backend_sp_db', exportdirs, )

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)



In [4]:
conn = sqlite3.connect("Database/project_SP.db")

In [5]:
conn.execute('''CREATE TABLE IF NOT EXISTS SP_Project
             (ID INT PRIMARY KEY     NOT NULL);''')

In [26]:
import sqlite3
import pandas as pd

# Create a connection to the database
conn = sqlite3.connect("Database/project_SP.db")

# List of months
months = ["Januar", "Februar", "März", "April", "Mai", "Juni"]

# Create an empty DataFrame to store the sums
df_sum = pd.DataFrame(columns=['Month'])

# Iterate over the months
for month in months:
    # Import data from Excel
    df_cust = pd.read_excel(f"Imports/{month}_21.xlsx")

    # Calculate the sum of the values
    total_sum_ÜS = round(df_cust['Saldo ÜS'].sum(), 2)
    total_sum_GLAZ = round(df_cust['Saldo GLAZ'].sum(), 2)
    total_sum_Covid = round(df_cust['Saldo Covid'].sum(), 2)
    total_sum_Ist = round(df_cust['Ist'].sum(), 2)
    total_sum_Soll = round(df_cust['Soll'].sum(),2)
    total_sum_DIS = round(df_cust['Differenz Ist/Soll'].sum(),2)
    total_sum_Ferien = round(df_cust['(5010) Ferien - Saldo'].sum(),2)

    # Append the sum to the DataFrame
    df_sum = df_sum.append({'Month': month, 'Saldo Überstunden': total_sum_ÜS, 'Saldo Glaz': total_sum_GLAZ, 'Saldo Covid': total_sum_Covid, 'Saldo IST': total_sum_Ist
    , 'Saldo SOLL': total_sum_Soll,'Saldo DIS': total_sum_DIS, 'Saldo Ferien': total_sum_Ferien }, ignore_index=True)


# Create a new table with the sums
df_sum.to_sql("SP_Project_Sums", conn, if_exists="replace", index=False)

# Commit the changes to the database
conn.commit()

# Close the connection
conn.close()


In [ ]:
conn = sqlite3.connect("Database/project_SP.db")

df_cust = pd.read_sql(""" SELECT
                      SUM(Jan.[Saldo ÜS] + Feb.[Saldo ÜS]+ März.[Saldo ÜS]) FROM SP_Project_Januar AS Jan, SP_Project_Februar AS Feb, SP_Project_März AS März  """)